# **Facial Attribute Classification for Personalized Emoji Generation**
personalized emoji generation engine 📷 😀

Library Imports

In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import torchvision.models as models
import torch.nn as nn
import PIL
from PIL import Image
from torchvision import transforms
import torch.optim as optim
import numpy as np
import os
import zipfile

In [ ]:
#Loading the labelled data into pandas data frame.
attribute_label_df = pd.read_csv("/kaggle/input/celeba-dataset/list_attr_celeba.csv", delim_whitespace=True,dtype='object',header=None)
attribute_label_df.head()

,0
0,"image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attr..."
1,"000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1..."
2,"000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,1,-1..."
3,"000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-..."
4,"000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,..."


## 40 Facial Attribute Classification train part
change to your attribute file path and img_align_celeba directory

In [ ]:
################################################
##train
################################################
loss_list = []
accuracy_list = []

#Loading the labelled data into pandas data frame.
attribute_label_df = pd.read_csv("/kaggle/input/celeba-updated/list_attr_celeba.txt", delim_whitespace=True,dtype='object',header=None)
attribute_label_df.head()

print("############################# Start Loading Data ##############################")
#Splitting the labelled data into train, validate and test.
train, temp = train_test_split(attribute_label_df, test_size=0.2,shuffle=False)
validate,test = train_test_split(temp, test_size=0.5,shuffle=False)

print(len(train.index))
print(len(validate.index))
print(len(test.index))
print("############################# Completed Loading Data ##############################")


#Checking if cuda is present. Assigning cuda to device.
print(torch.cuda.is_available())
if torch.cuda.is_available():
    device=torch.device('cuda')
    print("The device is" +str(device))
print("############################# Assigned Device ##############################")

torch.cuda.empty_cache()

# Loading pretrained ResNext model.
resnext50_32x4d = models.resnext50_32x4d(pretrained=False)  # Set pretrained to False
# Load the state dictionary from the downloaded file
state_dict = torch.load("/kaggle/input/resnext50/resnext50_32x4d-7cdf4587.pth", map_location=device)
resnext50_32x4d.load_state_dict(state_dict)

resnext50_32x4d.fc = nn.Linear(2048, 40)

#The first 5 layers of the ResNext is frozen using param.requires_grad = False
ct = 0
for child in resnext50_32x4d.children():
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

#Senting the model to GPU
resnext50_32x4d.to(device)
print("############################# Created Model ##############################")


#To calculate the accuracy persentage of the predicted data.
def get_accuracy( scores , labels ):
    accuracy=torch.FloatTensor(1,40)
    bs=scores.size(0)
    for j in range(0,40):
        correctCnt = 0
        for i in range(0,bs):
            if(scores[i,j]==labels[i,j]):
                correctCnt=correctCnt+1
        accuracy[0,j]=(correctCnt/bs)*100
    return accuracy


total_training_Samples = len(train.index)
default_bs=50

#Intialing loss function to MSE loss
criterion=nn.MSELoss()

#Initializing the optimizer. Mentioning to process the layers whose requires_grad is True.
optimizer = optim.Adamax(filter(lambda p: p.requires_grad, resnext50_32x4d.parameters()))

#Converting the label data from data frame to tensor.
image_label_ToCompare = torch.Tensor(np.array(pd.DataFrame(train.drop(train.columns[0],axis=1)).values).astype('float64')).float()

for epoch in range(0,5):
    total_Batches = int(total_training_Samples/default_bs) + 1
    for minibatch in range(0,total_Batches):
        torch.cuda.empty_cache()
        if(minibatch==(total_Batches-1)):
            bs=total_training_Samples%default_bs
        else:
            bs=default_bs

        batch_imageTensor = torch.cuda.FloatTensor(bs*3,3,218,178)
        minibatch_label = torch.cuda.FloatTensor(bs*3,40)
        for imageNumber in range(minibatch*default_bs,(minibatch*default_bs)+bs):

            #Loading Image
            imageName = train[0][imageNumber]
            pil_img = Image.open("/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/"+imageName).convert('RGB')
            #Transform Image to tensor
            batch_imageTensor[imageNumber - (minibatch*default_bs)] = transforms.ToTensor()(pil_img)
            minibatch_label[imageNumber - (minibatch*default_bs)] = image_label_ToCompare[imageNumber]
            #Image augmenataion - horizontal flip.
            pil_img=pil_img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
            batch_imageTensor[(imageNumber - (minibatch*default_bs))+bs] = transforms.ToTensor()(pil_img)
            minibatch_label[(imageNumber - (minibatch*default_bs))+bs] = image_label_ToCompare[imageNumber]
            #Image augmenataion - resizing and cropping
            pil_img=pil_img.resize((220,270))
            pil_img=pil_img.crop((21,26,199,244))
            batch_imageTensor[(imageNumber - (minibatch*default_bs))+(2*bs)] = transforms.ToTensor()(pil_img)
            minibatch_label[(imageNumber - (minibatch*default_bs))+(2*bs)] = image_label_ToCompare[imageNumber]

        #Senting input to GPU
        batch_imageTensor.to(device)
        #Senting label values to GPU
        minibatch_label.to(device)

        #Setting the gradients to 0 to remove all previous calculations.
        optimizer.zero_grad()
        #Predicting in input data
        scores=resnext50_32x4d(batch_imageTensor)

        #Calculating the loss
        loss =  criterion(scores,minibatch_label)
        #Calculating the gradients.
        loss.backward()
        #Updating the weights using the loss gradient.
        optimizer.step()

        #Applying threshold to convert contious to discrete values.
        converted_Score=scores.clone()
        converted_Score[converted_Score>=0]=1
        converted_Score[converted_Score<0]=-1

        #Calculating accuracy
        accuracy=get_accuracy(converted_Score,minibatch_label)

        #deleting tensors from GPU to save memmory
        del batch_imageTensor
        del minibatch_label
        #Clearing GPU cache
        torch.cuda.empty_cache()

        #Calculating and logging the accuracy and Loss
        if minibatch % 50 == 0:
            print("@Loss in epoch " + str(epoch) + " in minibatch " + str(minibatch) + " is " + str(loss.detach().item()))
            print("#Accuracy tensor of features is " + str(accuracy))
            print("$Net Accuracy is " + str(torch.sum(accuracy) / 40) + " %")
        #print("@Loss in the epoch "+str(epoch)+" in the minibatch "+str(minibatch)+ " is "+ str(loss.detach().item()))
        #print("#Accuracy tensor of features is "+str(accuracy))
        #print("$Net Accuracy is"+ str(torch.sum(accuracy)/40) +" %")

        ####


    #Saving the model parameters of the currect epoch.

    path_toSave = "/kaggle/working/"+str(epoch)+"_epoch.pt"
    torch.save({
            'model_state_dict': resnext50_32x4d.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, path_toSave)
    loss_list.append(loss.detach().item())
    accuracy_list.append(torch.sum(accuracy) / 40)
    print("#############################  Model Saved   ##############################")

In [ ]:
print(loss_list)
print(accuracy_list)

## 40 Facial Attribute Classification validation part
change to your attribute file path and img_align_celeba directory

In [ ]:
#################################################
## Validation
################################################

#Loading the labelled data into pandas data frame
attribute_label_df = pd.read_csv("/kaggle/input/celeba-updated/list_attr_celeba.txt", delim_whitespace=True,dtype='object',header=None)
attribute_label_df.head()

print("############################# Start Loading Data ##############################")
#Splitting the labelled data into train, validate and test.
train, temp = train_test_split(attribute_label_df, test_size=0.2,shuffle=False)
validate,test = train_test_split(temp, test_size=0.5,shuffle=False)

print(len(train.index))
print(len(validate.index))
print(len(test.index))
print("############################# Completed Loading Data ##############################")


#Checking if cuda is present. Assigning cuda to device.
print(torch.cuda.is_available())
if torch.cuda.is_available():
    device=torch.device('cuda')
    print("The device is" +str(device))
print("############################# Assigned Device ##############################")

#To calculate the accuracy persentage of the predicted data.
def get_accuracy( scores , labels ):
    accuracy=torch.FloatTensor(1,40)
    bs=scores.size(0)
    for j in range(0,40):
        correctCnt = 0
        for i in range(0,bs):
            if(scores[i,j]==labels[i,j]):
                correctCnt=correctCnt+1
        accuracy[0,j]=(correctCnt/bs)*100
    return accuracy


total_validate_Samples = len(validate.index)
startValidateIndex=int(validate.index[0])
default_bs=50
#Converting the label data from data frame to tensor.
image_label_Tovalidate = torch.Tensor(np.array(pd.DataFrame(validate.drop(validate.columns[0],axis=1)).values).astype('float64')).float()


torch.cuda.empty_cache()

# Loading pretrained ResNext model.
resnext50_32x4d = models.resnext50_32x4d(pretrained=False)  # Set pretrained to False
# Load the state dictionary from the downloaded file
state_dict = torch.load("/kaggle/input/resnext50/resnext50_32x4d-7cdf4587.pth", map_location=device)
resnext50_32x4d.load_state_dict(state_dict)

resnext50_32x4d.fc = nn.Linear(2048, 40)

#The first 5 layers of the ResNext is frozen using param.requires_grad = False
ct = 0
for child in resnext50_32x4d.children():
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

#Senting the model to GPU
resnext50_32x4d.to(device)

#Intialing loss function to MSE loss
criterion=nn.MSELoss()

#Iterate over all the epoch model and fing their accuracy in validation data.
for model in range(0,4):

    #Loading the check point details
    path_toLoad = "/kaggle/input/train-modle/model_"+str(model)+"_epoch.pt"
    checkpoint = torch.load(path_toLoad)

    #Initializing the model with the model parameters of the checkpoint.
    resnext50_32x4d.load_state_dict(checkpoint['model_state_dict'])
    #Setting the model to be in evaluation mode. This will set the batch normalization parameters.
    resnext50_32x4d.eval()

    total_Batches = int(total_validate_Samples/default_bs) + 1
    runningAccuracyOfModel=0
    featureAccuracy=torch.DoubleTensor(1,40)

    for minibatch in range(0,total_Batches):
        torch.cuda.empty_cache()
        if(minibatch==(total_Batches-1)):
            bs=total_validate_Samples%default_bs
        else:
            bs=default_bs

        batch_imageTensor = torch.cuda.FloatTensor(bs,3,218,178)
        minibatch_label = torch.cuda.FloatTensor(bs,40)
        for imageNumber in range((minibatch*default_bs)+startValidateIndex,(minibatch*default_bs)+bs+startValidateIndex):
            #Loading image and converting to tensor.
            imageName = validate[0][imageNumber]
            pil_img = Image.open("/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/"+imageName).convert('RGB')
            batch_imageTensor[imageNumber - ((minibatch*default_bs)+startValidateIndex)] = transforms.ToTensor()(pil_img)
            minibatch_label[imageNumber - ((minibatch*default_bs)+startValidateIndex)] = image_label_Tovalidate[imageNumber-startValidateIndex]

        batch_imageTensor.to(device)
        minibatch_label.to(device)

        #Doing predinction
        scores=resnext50_32x4d(batch_imageTensor)

        #Finding the loss
        loss =  criterion(scores,minibatch_label)

        #Applying threshold on the prediction to convert to binary values.
        converted_Score=scores.clone()
        converted_Score[converted_Score>=0]=1
        converted_Score[converted_Score<0]=-1

        #getting accuracy
        accuracy=get_accuracy(converted_Score,minibatch_label)

        del batch_imageTensor
        del minibatch_label
        torch.cuda.empty_cache()

        print("accuracy tensor in batch "+str(minibatch) + " of model " +str(model) +" is "+ str(accuracy))
        print("Net accuracy of batch "+ str(minibatch) + " of model " +str(model) +" is "+ str(torch.sum(accuracy)/40) +" %")
        print("Loss of the batch "+str(minibatch) + " of model " +str(model) +" is "+ str(loss.detach().item()))

        #Calculate running accuracy.
        runningAccuracyOfModel+=torch.sum(accuracy)/40
        featureAccuracy+=accuracy

    print("@@Net accuracy of model "+str(model)+" = "+str(runningAccuracyOfModel/total_Batches))
    print("@@Feature wise net accuracy of model "+str(model)+" = "+str(featureAccuracy/total_Batches))

## 40 Facial Attribute Classification test part
change to your attribute file path and img_align_celeba directory

In [ ]:
############
##test
############
#Loading the labelled data into pandas data frame
attribute_label_df = pd.read_csv("/kaggle/input/celeba-updated/list_attr_celeba.txt", delim_whitespace=True,dtype='object',header=None)
attribute_label_df.head()

print("############################# Start Loading Data ##############################")
#Splitting the labelled data into train, validate and test.
train, temp = train_test_split(attribute_label_df, test_size=0.2,shuffle=False)
validate,test = train_test_split(temp, test_size=0.5,shuffle=False)

print(len(train.index))
print(len(validate.index))
print(len(test.index))
print("############################# Completed Loading Data ##############################")


#Checking if cuda is present. Assigning cuda to device.
print(torch.cuda.is_available())
if torch.cuda.is_available():
    device=torch.device('cuda')
    print("The device is" +str(device))
print("############################# Assigned Device ##############################")

#To calculate the accuracy persentage of the predicted data.
def get_accuracy( scores , labels ):
    accuracy=torch.FloatTensor(1,40)
    bs=scores.size(0)
    for j in range(0,40):
        correctCnt = 0
        for i in range(0,bs):
            if(scores[i,j]==labels[i,j]):
                correctCnt=correctCnt+1
        accuracy[0,j]=(correctCnt/bs)*100
    return accuracy


total_test_Samples = len(test.index)
startTestIndex=int(test.index[0])
default_bs=50
#Converting the label data from data frame to tensor.
image_label_Totest = torch.Tensor(np.array(pd.DataFrame(test.drop(test.columns[0],axis=1)).values).astype('float64')).float()

#Describing the model architecture to initialise with data from checkpoint
# Loading pretrained ResNext model.
resnext50_32x4d = models.resnext50_32x4d(pretrained=False)  # Set pretrained to False
# Load the state dictionary from the downloaded file
state_dict = torch.load("/kaggle/input/resnext50/resnext50_32x4d-7cdf4587.pth", map_location=device)
resnext50_32x4d.load_state_dict(state_dict)

resnext50_32x4d.fc = nn.Linear(2048, 40)

#The first 5 layers of the ResNext is frozen using param.requires_grad = False
ct = 0
for child in resnext50_32x4d.children():
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

#Senting the model to GPU
resnext50_32x4d.to(device)

#Loading the checkpoint of the model that gave highest accuracy for the validation test data set.
path_toLoad = "/kaggle/input/train-modle/3_epoch.pt.pt"
checkpoint = torch.load(path_toLoad)


#Initializing the model with the model parameters of the checkpoint.
resnext50_32x4d.load_state_dict(checkpoint['model_state_dict'])
#Setting the model to be in evaluation mode. This will set the batch normalization parameters.
resnext50_32x4d.eval()

total_Batches = int(total_test_Samples/default_bs)
if(total_test_Samples%default_bs>0):
    total_Batches=total_Batches+1
runningAccuracyOfModel=0
featureAccuracy=torch.DoubleTensor(1,40)

for minibatch in range(0,total_Batches):
    torch.cuda.empty_cache()
    if(minibatch==(total_Batches-1)):
        bs=total_test_Samples%default_bs
    else:
        bs=default_bs

    batch_imageTensor = torch.cuda.FloatTensor(bs,3,218,178)
    minibatch_label = torch.cuda.FloatTensor(bs,40)
    for imageNumber in range((minibatch*default_bs)+startTestIndex,(minibatch*default_bs)+bs+startTestIndex):
        #Loading image and converting to tensor.
        imageName = test[0][imageNumber]
        pil_img = Image.open("/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/"+imageName).convert('RGB')
        batch_imageTensor[imageNumber - ((minibatch*default_bs)+startTestIndex)] = transforms.ToTensor()(pil_img)
        minibatch_label[imageNumber - ((minibatch*default_bs)+startTestIndex)] = image_label_Totest[imageNumber-startTestIndex]

    batch_imageTensor.to(device)
    minibatch_label.to(device)
    #Doing prediction on test data
    scores=resnext50_32x4d(batch_imageTensor)

    #Applying threshold
    converted_Score=scores.clone()
    converted_Score[converted_Score>=0]=1
    converted_Score[converted_Score<0]=-1

    #Calculating accuracy
    accuracy=get_accuracy(converted_Score,minibatch_label)

    del batch_imageTensor
    del minibatch_label
    torch.cuda.empty_cache()

    print( "accuracy tensor in batch "+str(minibatch) +" is "+ str(accuracy))
    print("Net accuracy of batch "+ str(minibatch) +" is "+ str(torch.sum(accuracy)/40) +" %")

    #Calculate running accuracy
    runningAccuracyOfModel+=torch.sum(accuracy)/40
    featureAccuracy+=accuracy

print("@@accuracy of model  = "+str(runningAccuracyOfModel/total_Batches))
print("@@Feature wise accuracy of model = "+str(featureAccuracy/total_Batches))

**Cololr tuning part**

## Skin tone Classification train part
change to your updated_color_celeba.txt attribute file path and img_align_celeba directory

In [ ]:
################################################
##Skin tone Train
################################################
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import torchvision.models as models
import torch.nn as nn
import PIL
from PIL import Image
from torchvision import transforms
import torch.optim as optim
import numpy as np

#Loading the labelled data into pandas data frame.
attribute_label_df = pd.read_csv("/kaggle/input/only-color-dataset/updated_color_celeba.txt", delim_whitespace=True,dtype='object',header=None)
attribute_label_df.head()

print("############################# Start Loading Data ##############################")
#Splitting the labelled data into train, validate and test.
train, temp = train_test_split(attribute_label_df, test_size=0.4,shuffle=False)
validate,test = train_test_split(temp, test_size=0.5,shuffle=False)

print(len(train.index))
print(len(validate.index))
print(len(test.index))
print("############################# Completed Loading Data ##############################")


#Checking if cuda is present. Assigning cuda to device.
print(torch.cuda.is_available())
if torch.cuda.is_available():
    device=torch.device('cuda')
    print("The device is" +str(device))
print("############################# Assigned Device ##############################")

torch.cuda.empty_cache()
#######################################################################################

# Loading pretrained ResNext model.
resnext50_32x4d = models.resnext50_32x4d(pretrained=False)  # Set pretrained to False
# Load the state dictionary from the downloaded file
state_dict = torch.load("/kaggle/input/resnext50/resnext50_32x4d-7cdf4587.pth", map_location=device)
resnext50_32x4d.load_state_dict(state_dict)

resnext50_32x4d.fc = nn.Linear(2048, 40)

#The first 5 layers of the ResNext is frozen using param.requires_grad = False
ct = 0
for child in resnext50_32x4d.children():
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

#Senting the model to GPU
resnext50_32x4d.to(device)

# Loading the checkpoint of the model that gave highest accuracy for the validation test data set.
path_toLoad = "/kaggle/input/train-modle/3_epoch.pt"
checkpoint = torch.load(path_toLoad, map_location=device)

# Initializing the model with the model parameters of the checkpoint.
resnext50_32x4d.load_state_dict(checkpoint['model_state_dict'])


#######################################################################################
#replacing the model head
resnext50_32x4d.fc = nn.Linear(2048, 5)
"""
print("############################# Created Model ##############################")


#The first 5 layers of the ResNext is frozen using param.requires_grad = False
ct = 0
for child in resnext50_32x4d.children():
    ct += 1
    if ct < 10:
        for param in child.parameters():
            param.requires_grad = False
"""

# Set requires_grad=False for all parameters except the ones in the new layer
for param in resnext50_32x4d.parameters():
    param.requires_grad = False

for param in resnext50_32x4d.fc.parameters():
    param.requires_grad = True


#Senting the model to GPU
resnext50_32x4d.to(device)
print("############################# Created Model ##############################")


#To calculate the accuracy persentage of the predicted data.
def get_accuracy( scores , labels ):
    accuracy=torch.FloatTensor(1,5)
    bs=scores.size(0)
    for j in range(0,5):
        correctCnt = 0
        for i in range(0,bs):
            if(scores[i,j]==labels[i,j]):
                correctCnt=correctCnt+1
        accuracy[0,j]=(correctCnt/bs)*100
    return accuracy


total_training_Samples = len(train.index)
default_bs=2

#Intialing loss function to MSE loss
criterion=nn.MSELoss()

#Initializing the optimizer. Mentioning to process the layers whose requires_grad is True.
optimizer = optim.Adamax(filter(lambda p: p.requires_grad, resnext50_32x4d.parameters()))

#Converting the label data from data frame to tensor.
image_label_ToCompare = torch.Tensor(np.array(pd.DataFrame(train.drop(train.columns[0],axis=1)).values).astype('float64')).float()


for epoch in range(0,100):
    total_Batches = int(total_training_Samples/default_bs) + 1
    for minibatch in range(0,total_Batches):
        torch.cuda.empty_cache()
        bs=default_bs

        batch_imageTensor = torch.cuda.FloatTensor(bs*3,3,218,178)
        minibatch_label = torch.cuda.FloatTensor(bs*3,5)
        for imageNumber in range(minibatch*default_bs,(minibatch*default_bs)+bs):

            #Loading Image
            #imageName = train[0][imageNumber]
            if imageNumber < len(train):
                imageName = train[0][imageNumber]

                pil_img = Image.open("/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/"+imageName).convert('RGB')
                #Transform Image to tensor
                batch_imageTensor[imageNumber - (minibatch*default_bs)] = transforms.ToTensor()(pil_img)
                minibatch_label[imageNumber - (minibatch*default_bs)] = image_label_ToCompare[imageNumber]
                #Image augmenataion - horizontal flip.
                pil_img=pil_img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
                batch_imageTensor[(imageNumber - (minibatch*default_bs))+bs] = transforms.ToTensor()(pil_img)
                minibatch_label[(imageNumber - (minibatch*default_bs))+bs] = image_label_ToCompare[imageNumber]
                #Image augmenataion - resizing and cropping
                pil_img=pil_img.resize((220,270))
                pil_img=pil_img.crop((21,26,199,244))
                batch_imageTensor[(imageNumber - (minibatch*default_bs))+(2*bs)] = transforms.ToTensor()(pil_img)
                minibatch_label[(imageNumber - (minibatch*default_bs))+(2*bs)] = image_label_ToCompare[imageNumber]
            else:
                print(f"Invalid imageNumber: {imageNumber}. Skipping this iteration.")

        #Senting input to GPU
        batch_imageTensor.to(device)
        #Senting label values to GPU
        minibatch_label.to(device)

        #Setting the gradients to 0 to remove all previous calculations.
        optimizer.zero_grad()
        #Predicting in input data
        scores=resnext50_32x4d(batch_imageTensor)

        #Calculating the loss
        loss =  criterion(scores,minibatch_label)
        #Calculating the gradients.
        loss.backward()
        #Updating the weights using the loss gradient.
        optimizer.step()

        #Applying threshold to convert contious to discrete values.
        converted_Score=scores.clone()
        converted_Score[converted_Score>=0]=1
        converted_Score[converted_Score<0]=-1



        #Calculating accuracy
        accuracy=get_accuracy(converted_Score,minibatch_label)

        #deleting tensors from GPU to save memmory
        del batch_imageTensor
        del minibatch_label
        #Clearing GPU cache
        torch.cuda.empty_cache()

        #Calculating and logging the accuracy and Loss
        #print("@Loss in the epoch "+str(epoch)+" in the minibatch "+str(minibatch)+ " is "+ str(loss.detach().item()))
        #print("#Accuracy tensor of features is "+str(accuracy))
        #print("$Net Accuracy is"+ str(torch.sum(accuracy)/5) +" %")

        ####


    #Saving the model parameters of the currect epoch.

    path_toSave = "/kaggle/working/"+str(epoch)+"_epoch-5-type.pt"
    torch.save({
            'model_state_dict': resnext50_32x4d.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, path_toSave)

    print("#############################  Model Saved   ##############################")

#Saving the model parameters of the currect epoch.

#path_toSave = "/kaggle/working/"+str(epoch)+"_epoch-5-mega.pt"
#torch.save({
        #'model_state_dict': resnext50_32x4d.state_dict(),
      #  'optimizer_state_dict': optimizer.state_dict()
      #  }, path_toSave)

## Skin tone Classification validation part
change to your updated_color_celeba.txt attribute file path and img_align_celeba directory

In [ ]:

#################################################
## Skin tone Validation
################################################

#Loading the labelled data into pandas data frame
attribute_label_df = pd.read_csv("/kaggle/input/only-color-dataset/only_color_celeba_mix.txt", delim_whitespace=True,dtype='object',header=None)
attribute_label_df.head()

print("############################# Start Loading Data ##############################")
#Splitting the labelled data into train, validate and test.
train, temp = train_test_split(attribute_label_df, test_size=0.2,shuffle=False)
validate,test = train_test_split(temp, test_size=0.5,shuffle=False)

print(len(train.index))
print(len(validate.index))
print(len(test.index))
print("############################# Completed Loading Data ##############################")


#Checking if cuda is present. Assigning cuda to device.
print(torch.cuda.is_available())
if torch.cuda.is_available():
    device=torch.device('cuda')
    print("The device is" +str(device))
print("############################# Assigned Device ##############################")

#To calculate the accuracy persentage of the predicted data.
def get_accuracy( scores , labels ):
    accuracy=torch.FloatTensor(1,5)
    bs=scores.size(0)
    for j in range(0,5):
        correctCnt = 0
        for i in range(0,bs):
            if(scores[i,j]==labels[i,j]):
                correctCnt=correctCnt+1
        accuracy[0,j]=(correctCnt/bs)*100
    return accuracy


total_validate_Samples = len(validate.index)
startValidateIndex=int(validate.index[0])
default_bs=1
#Converting the label data from data frame to tensor.
image_label_Tovalidate = torch.Tensor(np.array(pd.DataFrame(validate.drop(validate.columns[0],axis=1)).values).astype('float64')).float()


torch.cuda.empty_cache()

# Loading pretrained ResNext model.
resnext50_32x4d = models.resnext50_32x4d(pretrained=False)  # Set pretrained to False
# Load the state dictionary from the downloaded file
state_dict = torch.load("/kaggle/input/resnext50/resnext50_32x4d-7cdf4587.pth", map_location=device)
resnext50_32x4d.load_state_dict(state_dict)

resnext50_32x4d.fc = nn.Linear(2048, 5)

#The first 5 layers of the ResNext is frozen using param.requires_grad = False
ct = 0
for child in resnext50_32x4d.children():
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

#Senting the model to GPU
resnext50_32x4d.to(device)

#Intialing loss function to MSE loss
criterion=nn.MSELoss()

#Iterate over all the epoch model and fing their accuracy in validation data.
for model in range(0,30):

    #Loading the check point details
    path_toLoad = "/kaggle/working/"+str(model)+"_epoch-5-type.pt"
    checkpoint = torch.load(path_toLoad)

    #Initializing the model with the model parameters of the checkpoint.
    resnext50_32x4d.load_state_dict(checkpoint['model_state_dict'])
    #Setting the model to be in evaluation mode. This will set the batch normalization parameters.
    resnext50_32x4d.eval()

    total_Batches = int(total_validate_Samples/default_bs) + 1
    runningAccuracyOfModel=0
    featureAccuracy=torch.DoubleTensor(1,5)

    for minibatch in range(0,total_Batches):
        torch.cuda.empty_cache()
        bs=default_bs

        batch_imageTensor = torch.cuda.FloatTensor(bs,3,218,178)
        minibatch_label = torch.cuda.FloatTensor(bs,5)
        for imageNumber in range((minibatch*default_bs)+startValidateIndex,(minibatch*default_bs)+bs+startValidateIndex):
            #Loading image and converting to tensor.
            if imageNumber < len(validate):
                imageName = validate[0][imageNumber]
                pil_img = Image.open("/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/"+imageName).convert('RGB')
                batch_imageTensor[imageNumber - ((minibatch*default_bs)+startValidateIndex)] = transforms.ToTensor()(pil_img)
                minibatch_label[imageNumber - ((minibatch*default_bs)+startValidateIndex)] = image_label_Tovalidate[imageNumber-startValidateIndex]
            else:
                print(f"Invalid imageNumber: {imageNumber}. Skipping this iteration.")

        batch_imageTensor.to(device)
        minibatch_label.to(device)

        #Doing predinction
        scores=resnext50_32x4d(batch_imageTensor)

        #Finding the loss
        loss =  criterion(scores,minibatch_label)

        #Applying threshold on the prediction to convert to binary values.
        converted_Score=scores.clone()
        converted_Score[converted_Score>=0]=1
        converted_Score[converted_Score<0]=-1

        #getting accuracy
        accuracy=get_accuracy(converted_Score,minibatch_label)

        del batch_imageTensor
        del minibatch_label
        torch.cuda.empty_cache()

        print("accuracy tensor in batch "+str(minibatch) + " of model " +str(model) +" is "+ str(accuracy))
        print("Net accuracy of batch "+ str(minibatch) + " of model " +str(model) +" is "+ str(torch.sum(accuracy)/5) +" %")
        print("Loss of the batch "+str(minibatch) + " of model " +str(model) +" is "+ str(loss.detach().item()))

        #Calculate running accuracy.
        runningAccuracyOfModel+=torch.sum(accuracy)/5
        featureAccuracy+=accuracy

    print("@@Net accuracy of model "+str(model)+" = "+str(runningAccuracyOfModel/total_Batches))
    print("@@Feature wise net accuracy of model "+str(model)+" = "+str(featureAccuracy/total_Batches))



## Skin tone Classification test part
change to your updated_color_celeba.txt attribute file path and img_align_celeba directory

In [ ]:
################################################
## Skin tone test
################################################
#Loading the labelled data into pandas data frame
attribute_label_df = pd.read_csv("/kaggle/input/only-color-dataset/only_color_celeba_mix.txt", delim_whitespace=True,dtype='object',header=None)
attribute_label_df.head()

print("############################# Start Loading Data ##############################")
#Splitting the labelled data into train, validate and test.
train, temp = train_test_split(attribute_label_df, test_size=0.2,shuffle=False)
validate,test = train_test_split(temp, test_size=0.5,shuffle=False)

print(len(train.index))
print(len(validate.index))
print(len(test.index))
print("############################# Completed Loading Data ##############################")


#Checking if cuda is present. Assigning cuda to device.
print(torch.cuda.is_available())
if torch.cuda.is_available():
    device=torch.device('cuda')
    print("The device is" +str(device))
print("############################# Assigned Device ##############################")

#To calculate the accuracy persentage of the predicted data.
def get_accuracy( scores , labels ):
    accuracy=torch.FloatTensor(1,5)
    bs=scores.size(0)
    for j in range(0,5):
        correctCnt = 0
        for i in range(0,bs):
            if(scores[i,j]==labels[i,j]):
                correctCnt=correctCnt+1
        accuracy[0,j]=(correctCnt/bs)*100
    return accuracy


total_test_Samples = len(test.index)
startTestIndex=int(test.index[0])
default_bs=1
#Converting the label data from data frame to tensor.
image_label_Totest = torch.Tensor(np.array(pd.DataFrame(test.drop(test.columns[0],axis=1)).values).astype('float64')).float()

#Describing the model architecture to initialise with data from checkpoint
# Loading pretrained ResNext model.
resnext50_32x4d = models.resnext50_32x4d(pretrained=False)  # Set pretrained to False
# Load the state dictionary from the downloaded file
state_dict = torch.load("/kaggle/input/resnext50/resnext50_32x4d-7cdf4587.pth", map_location=device)
resnext50_32x4d.load_state_dict(state_dict)

resnext50_32x4d.fc = nn.Linear(2048, 5)

#The first 5 layers of the ResNext is frozen using param.requires_grad = False
ct = 0
for child in resnext50_32x4d.children():
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

#Senting the model to GPU
resnext50_32x4d.to(device)

#Loading the checkpoint of the model that gave highest accuracy for the validation test data set.
path_toLoad = "/kaggle/working/9_epoch-5-type.pt"
checkpoint = torch.load(path_toLoad)


#Initializing the model with the model parameters of the checkpoint.
resnext50_32x4d.load_state_dict(checkpoint['model_state_dict'])
#Setting the model to be in evaluation mode. This will set the batch normalization parameters.
resnext50_32x4d.eval()

total_Batches = int(total_test_Samples/default_bs)
if(total_test_Samples%default_bs>0):
    total_Batches=total_Batches+1
runningAccuracyOfModel=0
featureAccuracy=torch.DoubleTensor(1,5)

for minibatch in range(0,total_Batches):
    torch.cuda.empty_cache()
    bs=default_bs

    batch_imageTensor = torch.cuda.FloatTensor(bs,3,218,178)
    minibatch_label = torch.cuda.FloatTensor(bs,5)
    for imageNumber in range((minibatch*default_bs)+startTestIndex,(minibatch*default_bs)+bs+startTestIndex):
        #Loading image and converting to tensor.
        imageName = test[0][imageNumber]
        pil_img = Image.open("/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/"+imageName).convert('RGB')
        batch_imageTensor[imageNumber - ((minibatch*default_bs)+startTestIndex)] = transforms.ToTensor()(pil_img)
        minibatch_label[imageNumber - ((minibatch*default_bs)+startTestIndex)] = image_label_Totest[imageNumber-startTestIndex]

    batch_imageTensor.to(device)
    minibatch_label.to(device)
    #Doing prediction on test data
    scores=resnext50_32x4d(batch_imageTensor)

    #Applying threshold
    converted_Score=scores.clone()
    converted_Score[converted_Score>=0]=1
    converted_Score[converted_Score<0]=-1

    #Calculating accuracy
    accuracy=get_accuracy(converted_Score,minibatch_label)

    del batch_imageTensor
    del minibatch_label
    torch.cuda.empty_cache()

    print( "accuracy tensor in batch "+str(minibatch) +" is "+ str(accuracy))
    print("Net accuracy of batch "+ str(minibatch) +" is "+ str(torch.sum(accuracy)/5) +" %")

    #Calculate running accuracy
    runningAccuracyOfModel+=torch.sum(accuracy)/40
    featureAccuracy+=accuracy

print("@@accuracy of model  = "+str(runningAccuracyOfModel/total_Batches))
print("@@Feature wise accuracy of model = "+str(featureAccuracy/total_Batches))